In [1]:
""" Script to obtain all battery design parameters from BatPaC"""

# from Model.battery_design import battery_design as bd
import matplotlib.pyplot as plt
from battery_design import battery_design as bd
import sys

local_path = r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\batt_sust_model"  # ADD PATH
sys.path.insert(1, local_path)

path_batpac =r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model-1\example notebooks\case study publication example\battery_design_run\runAB\BatPaC 5.0 - 8March2022_AB.xlsm"




A = [
    101.8642746,  # A coefficient
    3.42293361,  # B coefficient
    0.373273116,  # C coefficient
    55,  # Motor power
    5,  # Modules per row
    4,  # rows of modules
    200,        # RANGE (wltp)
    [12, 6],  # cells per module [20mm cell, 40mm cell],
    475,  # power requirement auxilary (W)
    [3,2],#cells in parallel 20/40
    [1,1]#modules in parallel
]
B = [
    110.5783408,
    -1.99365982,
    0.456185601,
    150,
    5,  # Modules per row
    4,  # rows of modules
    322,
    [14, 7],
    575,
    [3,2],
    [1,1]
]

C = [135.9999272,
        3.732390689,
        0.406771986,
        150,
        5,  # Modules per row
        4,  # rows of modules
        411,
        [18, 9],
        715,
        [3,2],
        [1,1]
        ]

D = [208.0433242,
        2.591081139,
        0.473213265,
        209,
        5,  # Modules per row
    4,  # rows of modules
        460,
        [20, 10],
        775,
        [3,2],
        [1,1]
        ]


cathode_foil = [10, 14, 18]
anode_foil = [6, 10, 14]
separator = [(5, 0), (7, 0), (9, 0), (5, 2), (7, 2), (9, 3)]
segment = [A,B]
cathode_am = ['NMC333-G (Energy)', 'NMC532-G (Energy)', 'NMC622-G (Energy)', 'NMC811-G (Energy)',
                'NCA-G (Energy)', 'LFP-G (Energy)', '50%/50% NMC532/LMO - G', "LMO-G (Energy)"]
silicon_anode = [0, 5, 10]
cell_thickness = [20, 40]

battery_designs_params = {}
count = 0



In [2]:

print('Establishing parameter dictionary')
for vehicle_parameters in segment:
    for cf in cathode_foil:
        for af in anode_foil:
            for sf in separator:
                for cam in cathode_am:
                    for sil in silicon_anode:
                        for cd in cell_thickness:
                            if cd == 20:
                                cells_module = 0
                            else:
                                cells_module = 1

                            name = count
                            battery = bd.Battery_system(
                                vehicle_type="EV",
                                electrode_pair=cam,
                                silicon_anode=sil,
                                calculate_fast_charge='Yes',
                                max_charging_time=60,
                                max_charge_power=150,
                                available_energy=94,
                                cells_in_parallel=vehicle_parameters[9][cells_module],
                                modules_in_parallel=1,
                                A_coefficient=vehicle_parameters[0],
                                B_coefficient=vehicle_parameters[1],
                                C_coefficient=vehicle_parameters[2],
                                vehicle_range_miles=vehicle_parameters[6]
                                * 0.621371192,  # MUST BE MILES IN BATPAC
                                motor_power=vehicle_parameters[3],
                                cells_per_module=vehicle_parameters[7][cells_module],  # THIS IS Z
                                modules_per_row=vehicle_parameters[4],
                                rows_of_modules=vehicle_parameters[5],
                                positive_foil_thickness=cf,
                                negative_foil_thickness=af,
                                sep_film_thickness=sf[0],
                                sep_coat_thickness=sf[1],
                                cell_thickness_target=cell_thickness[cells_module],
                                power_auxillary=vehicle_parameters[8],
                                positive_max_thickness=74

                            ).parameter_dictionary()
                            count += 1

                            battery_designs_params[name] = battery


Establishing parameter dictionary


In [3]:
calculated_system = bd.solve_batpac_battery_system_multiple(
        path_batpac, 
        battery_designs_params, 
        visible=False,
        save_iterations= 100
    )



100%|██████████| 5184/5184 [9:23:23<00:00,  6.52s/it]  


In [10]:
len([x for x in calculated_system.keys() if calculated_system[x]['material_content_pack']['anode active material (synthetic graphite)'] != 0 ])

5184

In [8]:
calculated_system[34]['material_content_pack']

{'anode active material (SiO)': 2.2457212730429377,
 'anode active material (natural graphite)': 0,
 'anode active material (synthetic graphite)': 20.21149145738644,
 'anode binder (CMC)': 0.27498627833178824,
 'anode binder additive (SBR)': 0.1833241855545255,
 'anode carbon black': 0.0,
 'anode current collector Cu (10um)': 0,
 'anode current collector Cu (11um)': 0,
 'anode current collector Cu (12um)': 0,
 'anode current collector Cu (13um)': 0,
 'anode current collector Cu (14um)': 0,
 'anode current collector Cu (6um)': 11.876873135636107,
 'anode current collector Cu (7um)': 0,
 'anode current collector Cu (8um)': 0,
 'anode current collector Cu (9um)': 0,
 'battery jacket': 53.32139648834495,
 'battery jacket Al': 19.25753896224536,
 'battery jacket Fe': 33.03386816906931,
 'battery jacket insulation': 1.0299893570302854,
 'battery management system': 3.5776000000000003,
 'battery pack': 256.4047103901575,
 'busbar': 0.4643301804703562,
 'cathode active material (50%/50% NMC532